In [ ]:
from nbdev import *

ModuleNotFoundError: No module named 'nbdev'

In [ ]:
%nbdev_default_export app

UsageError: Line magic function `%nbdev_default_export` not found.


# Emmaus Walking Streamlit App

> This is the front-end application built in Streamlit (which I believe is not compatible with Jupyter notebooks).
> However `nbdev` converts this to a straight .py script by default - which should be possible to `streamlit run`.

In [ ]:
%nbdev_hide
from nbdev.showdoc import *
from fastcore.test import *

UsageError: Line magic function `%nbdev_hide` not found.


In [ ]:
%nbdev_export
import numpy as np
import pandas as pd
import datetime as dt
import streamlit as st
from streamlit_folium import folium_static
import folium
import os, io
import activityio as aio
from IPython.display import display
from dateutil.parser import parse
#import matplotlib


#from pandas_profiling import ProfileReport
#from streamlit_pandas_profiling import st_profile_report

UsageError: Line magic function `%nbdev_export` not found.


In [ ]:
%nbdev_export
DATA_FILE = 'TBA'
AUTHOR_INFO = 'AUTHOR: [Michael J. Booth](https://about.me/mjboothaus)'
APP_NAME = 'Emmaus Walking Mapping App'

UsageError: Line magic function `%nbdev_export` not found.


In [ ]:
WALK_NAME = ['Great North Walk', 'Great West Walk', 'Bondi to Manly']

In [ ]:
%nbdev_export
class SideBar:
    datasource = 'DATA: [TBA](http://TBA)'
    datasize = 0   # look to calculate this (in MB?) - TEST: Comment change
    author = AUTHOR_INFO
    title = 'Data details...'
    data_local = False
    start_date = dt.date.today()
    end_date = dt.date.today()
    selected_data = None
    walk_name = ''


def create_sidebar():
    WALK_NAME = ['Great North Walk', 'Great West Walk', 'Bondi to Manly']
    sb = SideBar()
    st.sidebar.markdown(sb.author)
    st.sidebar.markdown(sb.datasource)
    st.sidebar.info(sb.title)
    st.sidebar.markdown('Datasize: ' + str(sb.datasize))
    sb.walk_name = st.sidebar.selectbox('Choose a walk', WALK_NAME, 0)
    return sb

def create_mainscreen(APP_NAME):
    st.title(APP_NAME)

    # Folium mapping example: center on Sydney Opera House
    m = folium.Map(location=[-33.85719805, 151.21512338473752], zoom_start=15)

    # add marker for Opera House
    tooltip = "Sydney Opera House"
    folium.Marker(
        [-33.85719805, 151.21512338473752], popup="Sydney Opera House", tooltip=tooltip
    ).add_to(m)

    # call to render Folium map in Streamlit
    folium_static(m)

def load_walk_data(walk_name):
    fit_files = sorted(os.listdir('./data/'))

UsageError: Line magic function `%nbdev_export` not found.


In [ ]:
%nbdev_export
sb = create_sidebar()

try:
    # df = pd.read_csv(DATA_FILE)
    
    create_mainscreen(APP_NAME)
except Exception as e:
    st.write(e)
    st.write('ERROR: Unable to download data')

In [ ]:
# TODOs:
#
# Setup drop-down to choose the specific walk to be mapped